In [1]:
import rioxarray # for the extension to load
import xarray as xr

xds = xr.open_dataarray("../ggcmi/phase2_outputs/dataset/yield/pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T4_W0_N60_A0.nc4", decode_times=False)

# min_lon = -24.995
# min_lat = 25.05
# max_lon = 45.50
# max_lat = 71.55

# xdsc = xds.rio.clip_box(
#     minx=min_lat,
#     miny=min_lon,
#     maxx=max_lat,
#     maxy=max_lon,
# ).set_spatial_dims(x_dim='lat', y_dim='lon', inplace=False)

# print(xdsc)

In [11]:
import rioxarray
import geopandas

# ncFile = "./pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T4_W0_N60_A0.nc4"
# ncFile = "./Lower_48_states.tif"
rastFile = "./Lower_48_states.tif"

# geodf = geopandas.read_file(ncFile)
xds = rioxarray.open_rasterio(rastFile)
# clipped = xds.rio.clip(geodf.geometry.apply(mapping), geodf.crs)

In [12]:
xds

<xarray.DataArray (band: 1, y: 768, x: 1366)>
[1049088 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -125.0 -124.9 -124.9 ... -65.61 -65.57 -65.52
  * y            (y) float64 49.98 49.95 49.92 49.88 ... 24.62 24.58 24.55 24.52
    spatial_ref  int64 0
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [1]:
import xarray
import rioxarray
import geopandas
from shapely.geometry import mapping

xds = xarray.open_dataset(
    "./pdssat_agmerra_fullharm_yield_mai_global_annual_1980_2010_C360_T4_W0_N60_A0.nc4", decode_times=False)

xds = xds[['yield_mai']].transpose('time', 'lat', 'lon')

xds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
xds.rio.write_crs("EPSG:4326", inplace=True)
geodf = geopandas.read_file("./Lower_48_grid.shp")

clipped = xds.rio.clip(geodf.geometry.apply(mapping), geodf.crs)
clipped

<xarray.Dataset>
Dimensions:      (lat: 51, lon: 119, time: 31)
Coordinates:
  * lat          (lat) float64 49.75 49.25 48.75 48.25 ... 25.75 25.25 24.75
  * lon          (lon) float64 -124.8 -124.2 -123.8 ... -66.75 -66.25 -65.75
  * time         (time) float64 1.0 2.0 3.0 4.0 5.0 ... 27.0 28.0 29.0 30.0 31.0
    spatial_ref  int64 0
Data variables:
    yield_mai    (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan

In [2]:
clipped.to_netcdf("./demo.nc4")

In [3]:
xds = xarray.open_dataset("./demo.nc4", decode_times=False)
xds

<xarray.Dataset>
Dimensions:      (lat: 51, lon: 119, time: 31)
Coordinates:
  * lat          (lat) float64 49.75 49.25 48.75 48.25 ... 25.75 25.25 24.75
  * lon          (lon) float64 -124.8 -124.2 -123.8 ... -66.75 -66.25 -65.75
  * time         (time) float64 1.0 2.0 3.0 4.0 5.0 ... 27.0 28.0 29.0 30.0 31.0
Data variables:
    spatial_ref  int64 ...
    yield_mai    (time, lat, lon) float32 ...

In [5]:
xdF = xds.to_dataframe()
xdF = xdF.dropna(how='any')
xdF

spatial_ref  yield_mai
lat   lon     time                        
49.75 -122.25 1.0             0      4.712
              2.0             0      4.688
              3.0             0      4.800
              4.0             0      5.026
              5.0             0      3.516
...                         ...        ...
24.75 -97.75  27.0            0      0.042
              28.0            0      0.609
              29.0            0      1.202
              30.0            0      0.011
              31.0            0      0.087

[110732 rows x 2 columns]